<a href="https://colab.research.google.com/github/alexndremelyanov/_/blob/main/L3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")
!pip install category_encoders | grep -v 'already satisfied'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder

In [ ]:
inclass_csv = pd.read_csv("insclass_train.csv")

inclass_attributes = inclass_csv.drop('target', axis=1)
target = inclass_csv['target']

train_data, test_data, train_target, test_target = train_test_split(
  inclass_attributes,
  target,
  test_size=0.25,
  stratify=target,
  random_state=50
)

encoder = TargetEncoder(cols=["variable_1",
                              "variable_5",
                              "variable_20",
                              "variable_21",
                              "variable_22",
                              "variable_28"]).fit(train_data, train_target)

encoder.transform(train_data)
train_data = encoder.transform(train_data)
test_data = encoder.transform(test_data)

train_data = train_data.drop(["variable_7", "variable_9", "variable_15", 'variable_3'], axis=1)
test_data = test_data.drop(["variable_7", "variable_9", "variable_15", 'variable_3'], axis=1)
columns = train_data.select_dtypes(include='number').columns

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(train_data[columns])

train_data[columns] = imputer.transform(train_data[columns])
test_data[columns] = imputer.transform(test_data[columns])

scaler = MinMaxScaler()

scaler.fit(train_data[columns])

train_data[columns] = scaler.transform(train_data[columns])
test_data[columns] = scaler.transform(test_data[columns])

train_data["variable_0"] = np.ones((train_data.shape[0]))
test_data["variable_0"] = np.ones((test_data.shape[0]))

columns = train_data.select_dtypes(include='number').columns

train_target = np.array(train_target).reshape(-1, 1)
test_target = np.array(test_target).reshape(-1, 1)

In [ ]:
class Regression:
    def __init__(self, threshold = 0.5):
        self.a = np.zeros(2).reshape(1, 2)
        self.threshold = threshold

    def sigmoid(self, x):
        return 1/(1 + np.exp(-x @ self.a))

    def predict_probability(self, x):
        return self.sigmoid(x)

    def predict(self, x):
        return (self.predict_probability(x) > self.threshold).astype(int)

    def coefs(self):
        return self.a

    def log_likelihood(self, x, Y):
        return (Y * np.log(self.predict_probability(x) + 1e-7) + (1 - Y) * np.log(1 + 1e-7 - self.predict_probability(x))).mean()

    def cross_entropy(self, x, Y):
        return (-Y * np.log(self.predict_probability(x)) - (1 - Y) * np.log(1 - self.predict_probability(x))).mean()

    def accuracy(self, x, Y):
        return (self.predict(x) == Y).mean()

    def fit(self, x, Y, alpha = 0.01, epsylon = 0.01, max_steps = 1000, Rtype = "LL"):
        self.a = np.zeros(x.shape[1]).reshape(x.shape[1],1)
        steps, errors = [], []
        step, new_error = 0,0
        for _ in range(max_steps):
            if Rtype == "LL":
                new_error = self.log_likelihood(x, Y)
                dT_a = x.T @ (Y - self.predict_probability(x)) / x.shape[0]
                self.a += alpha * dT_a

            elif Rtype == "CE":
                new_error = self.cross_entropy(x, Y)
                dT_a = -x.T @ (Y - self.predict_probability(x)) / x.shape[0]
                self.a -= alpha*dT_a
            step += 1
            steps.append(step)
            errors.append(new_error)

        return steps, errors

In [ ]:
ALPHA = 0.01
ESPYLON = 0.01
MAX_STEPS = 1000
THRESHOLD = 0.2
regression = Regression(threshold=THRESHOLD)

In [ ]:
ll_steps , ll_errors = regression.fit(train_data[columns], train_target, alpha=ALPHA,epsylon=ESPYLON,max_steps=MAX_STEPS,Rtype="LL")
accuracy = regression.accuracy(test_data[columns], test_target)[0]

print(f"Точность метода градиентного подъема: {accuracy}")

Точность метода градиентного подъема: 0.9042058543802177


In [ ]:
ce_steps, ce_errors = regression.fit(train_data[columns], train_target, alpha=ALPHA,epsylon=ESPYLON,max_steps=MAX_STEPS,Rtype="CE")
accuracy = regression.accuracy(test_data[columns], test_target)[0]

print(f"Точность метода градиентного спуска: {accuracy}")

Точность метода градиентного спуска: 0.9042058543802177


График функции потерь градиентного спуска

In [ ]:
fig = plt.figure()

ax = fig.add_subplot()


ax.plot(ce_steps, ce_errors, label='ce')

ax.plot(ll_steps, ll_errors, label='ll')
handles, titles = ax.get_legend_handles_labels()

ax.set_xlabel('Шаг')
ax.set_ylabel('Значение ошибки')
ax.legend(handles, titles)